<a href="https://colab.research.google.com/github/anoted/genai-test/blob/main/Assignment_3_Nobel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Assignment 3: Create a recipe generator using RAG

```
NOTE: This code should be run with a HuggingFace account with a collab CPU runtime and the option `High Ram`clicked.

```

For this assignment, you'll be creating a RAG-based recipe genertion system using Alibaba's `KingNish/Qwen2.5-0.5b-Test-ft` LLM, a small but high-accuracy LLM that can be run on a CPU. To implement the RAG component, we'll be using the `LlamaIndex` library and, as our embedding model, `BAAI/bge-small-en-v1.5`

This code should be run with a HuggingFace account with a collab CPU runtime and the option `High Ram`clicked.

The recipe dataset we'll be using is `m3hrdadfi/recipe_nlg_lite`. The train split of the dataset will be used for the index and a portion of the test dataset will be used to qualitiatively evaluate the RAG system. You'll also be asked to compare these results qualitatively with vanilla `Qwen2.5-0.5b-Test-ft`, i.e., prompt the `Qwen` base LLM for recipes and compare these to the RAG system to determine if there are any benefits to using RAG for this task.

Your task is to

1. complete the code in each of the cells below by inserting your own code wherevver you see  `### WRITE YOUR CODE HERE ###` Also, add a comment wherever you see `### WRITE YOUR COMMENT HERE ###`  You'll likely want to consult the LlamaIndex documenetation to complete much of this code. The LlamaIndex docs on RAG can be found [here](https://docs.llamaindex.ai/en/stable/understanding/rag/)

2. run all of cells after completing the code and answer the questions at the end of the notebook.



In [1]:
# uncomment and run in your environment / on Colab, if you haven't installed these packages yet
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install sentence-transformers
!pip install datasets
!pip install llama-index
!pip install "transformers[torch]" "huggingface_hub[inference]"
from IPython.display import clear_output
clear_output()

In [2]:
# import packages
from datasets import load_dataset
import os
import pandas as pd
from llama_index.core import VectorStoreIndex, Settings, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer
import torch

In [3]:
# load dataset from HF
dataset = load_dataset("m3hrdadfi/recipe_nlg_lite")
# convert train split to pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

recipe_nlg_lite.py:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

The repository for m3hrdadfi/recipe_nlg_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/m3hrdadfi/recipe_nlg_lite.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/6118 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1080 [00:00<?, ? examples/s]

In [4]:
# Let's take a look at the data
dataset_df.head()

,uid,name,description,link,ner,ingredients,steps
0,dab8b7d0-e0f6-4bb0-aed9-346e80dace1f,pork chop noodle soup,we all know how satisfying it is to make great...,https://www.yummly.com/private/recipe/Pork-Cho...,"bone in pork chops, salt, pepper, vegetable oi...","3.0 bone in pork chops, salt, pepper, 2.0 tabl...",season pork chops with salt and pepper . heat ...
1,b03f346bf39efcbace5d30a8f962147c8c4c361f,5 ingredient almond cake with fresh berries,this simple almond cake is made with just five...,https://www.skinnytaste.com/5-ingredient-almon...,"large eggs, large egg whites, sugar, pure vani...","3 large eggs, 3 large egg whites, 2/3 cup suga...",position a rack in the middle of the oven and ...
2,89b49e742b2c1d234b83044c14d81155dfea7f19,shrimp cakes,"these light, pan seared shrimp cakes are moist...",https://www.skinnytaste.com/shrimp-cakes/,"peeled and deveined jumbo shrimp, plus 3 table...","1 pound peeled and deveined jumbo shrimp, 1 cu...",pat shrimp dry with a paper towel and place in...
3,5db9af50-63dc-4c5b-9db1-783cf96675d3,chili roasted okra,"chili roasted okra with okra, sesame oil, red ...",https://www.yummly.com/private/recipe/Chili-Ro...,"okra, sesame oil, red pepper flakes, salt, pepper","1.0 pound okra, 1.0 tablespoon sesame oil, 1.0...",preheat the oven to 425degf . wash and dry the...
4,9b8da42d-d07c-4766-9f15-fd3fd6e19bf6,slow cooker chicken chili,warm up on a cold day with this slow cooker ch...,https://www.yummly.com/private/recipe/Slow-Coo...,"oil, chicken, chili powder, onion, jalapeno pe...","1.0 tablespoon oil, 1.0 pound chicken, 1.5 tab...",heat oil in skillet over medium high heat . ad...


In [5]:
# We'll crate a VectorStorageIndex with the texts from the train dataset. These will be formatted as
#"Name of recipe \n\n ingredients \n\n steps"

texts = [
    f"{row['name']} \n\n {row['ingredients']} \n\n {row['steps']}" for _, row in dataset_df.iterrows()
]
texts[:2]

['pork chop noodle soup \n\n 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery \n\n season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons salt and 1 teaspoon pepper . mix well and bring to a simmer . add chops back to pot and return to simmer . reduce heat and simmer for 90 minutes, stirring occasionally, being careful not to break up chops . transfer chops to plate, 

In [6]:
#We then load the texts into LlamaIndex's Document object. Later we'll load these into a vector database
documents = [Document(text=t) for t in texts]
documents[0]

Document(id_='5336f71d-1fdc-4b41-be7b-871598bcbe71', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='pork chop noodle soup \n\n 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery \n\n season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a s

In [7]:
# Create a utility function to format prompts. This will only need to be edited if you use something other than Qwen. If you
# decide to use something other than Qwen, check the HuggingFace model card for that model to determine prompt formatting
def completion_to_prompt(completion):
    return f"{completion}"

### Commented out to make it easier to run (uncomment if module missing)

In [8]:
!pip install accelerate # Using `bitsandbytes` 8-bit quantization requires Accelerate
!pip install -U bitsandbytes
clear_output()

In [9]:
# Save the setting reused by our RAG system across queries and specify the embedding model--
# the model we'll be using is `BAAI/bge-small-en-v1.5` but you're welcome to use another
Settings.embed_model = HuggingFaceEmbedding(
    #############################################
    model_name="BAAI/bge-small-en-v1.5",
    device="cuda" if torch.cuda.is_available() else "cpu"
    #############################################
)

# pass the LLM to our the settings object
# see these docs for more details: https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/
# and https://docs.llamaindex.ai/en/stable/module_guides/models/llms/usage_custom/
Settings.llm = HuggingFaceLLM(
    #############################################
    model_name="KingNish/Qwen2.5-0.5b-Test-ft",
    tokenizer_name="KingNish/Qwen2.5-0.5b-Test-ft",
    context_window=1024,
    max_new_tokens=128,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
    # Explicitly set load_in_8bit=False or remove it -- up to you
    model_kwargs={"torch_dtype": torch.float16, "trust_remote_code": True}  # "load_in_8bit": True
    #############################################
)
print("Set the LLM as KingNish/Qwen2.5-0.5b-Test-ft...")

# Now we create a vector store which converts the documents to Node objects
# (https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/)
print("Creating index...")
index = VectorStoreIndex.from_documents(
    #############################################
    documents # using base vector indexing
    #############################################
)
print("Done.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Set the LLM as KingNish/Qwen2.5-0.5b-Test-ft...
Creating index...
Done.


In [10]:
# https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/
# we define the query engine: generic interface that allows to ask questions over data
query_engine = index.as_query_engine(
    ### WRITE YOUR COMMENT HERE: What does compact do?###
    #######################################################
    # According to https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/
    # "compact" puts as many text chunks as possible into the LLM call at an instance (resulting in less LLM calls overall)

    # ## Directly copied from the documentation below
    # compact (default): similar to refine but compact (concatenate) the chunks beforehand, resulting in less LLM calls.
    # stuff as many text (concatenated/packed from the retrieved chunks) that can fit within the context window (considering the maximum prompt size between text_qa_template and refine_template). If the text is too long to fit in one prompt, it is split in as many parts as needed (using a TokenTextSplitter and thus allowing some overlap between text chunks).
    # Each text part is considered a "chunk" and is sent to the refine synthesizer.
    # In short, it is like refine, but with less LLM calls.
    #######################################################

    response_mode="compact",
    ### WRITE YOUR COMMENT HERE: What does similarity_top_k specify?###
    #######################################################
    # According to https://docs.llamaindex.ai/en/stable/understanding/indexing/indexing/#top-k-retrieval
    # It sets the top k (number) most similar indexed texts as embeddings to the query to be retrieved
    #######################################################
    similarity_top_k=3,
    verbose=True,
)
# https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/
response = query_engine.query("How do I make creme brulee?")
print(response)

for i, n in enumerate(response.source_nodes):
    print(f"----- Node {i} -----")
    print(n.node.get_content())
    print("score")
    print(n.score)

1. Preheat your oven to 325 degrees Celsius (600 degrees Fahrenheit). 2. In a saucepan, heat the cream and milk together. Cut the vanilla pod in half, and use a knife to remove seeds, and add to the milk and cream. Bring to a boil, and remove from heat. Separate the yolks from the whites, add sugar to the yolks, and gently add the hot cream, stirring gently. Do not overmix to avoid creating too many bubbles. 3. Preheat the oven to 325 degrees Celsius (600 degrees Fahrenheit). Divide the cream
----- Node 0 -----
rachel khoo's creme brulee 

 none none, 300.0 milliliter cream, 200.0 milliliter milk, 1.0 vanilla pod, 6.0 egg yolks, 100.0 gram sugar, 30.0 gram white sugar, 30.0 gram brown sugar 

 in a saucepan, heat the cream and milk . cut the vanilla pod in half, and use a knife to remove seeds, and add to the milk and cream . bring to a boil, and remove from heat . separate the yolks from the whites, add sugar to the yolks, and gently add the hot cream, stirring gently . do not over mi

In [11]:
# testing loop
rag_responses = []
vanilla_responses = []
retrieved_node_texts = []
retrieved_node_scores = []

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "KingNish/Qwen2.5-0.5b-Test-ft",
    device_map=device, #### added device - if possible run with gpu
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("KingNish/Qwen2.5-0.5b-Test-ft")

# retrieve 20 random dish names from test dataset to test the system on
test_df = pd.DataFrame(dataset["test"]).sample(20)
test_queries = [f'How do I make creme brulee?']
    #f'How do I make {r["name"]}?' for
    #_, r in test_df.iterrows()
#]
#print(test_queries[:5])


for query in test_queries:# you may want to just run a few of these via [:5]:
    ### WRITE YOUR CODE HERE ###
    # run the query against the RAG system
    ###################################################
    response_rag = query_engine.query(query)
    rag_responses.append(str(response_rag))
    ###################################################

    # get the texts of the nodes that were retrieved for this query as a list
    retrieved_node_texts.append(
        ###################################################
        [node.node.get_content() for node in response_rag.source_nodes]
        ###################################################
    )

    # get the scores of the texts of the retrieved nodes as a list
    retrieved_node_scores.append(
        ###################################################
        [node.score for node in response_rag.source_nodes]
        ###################################################
    )
    ### YOUR CODE HERE ###
    # implement the "vanilla" recipe generator, which simply prompts the base LLM (e.g., Qwen) rather than using Qwen+RAG
    input_text = completion_to_prompt(query)
    ###################################################
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    output = model.generate(
          input_ids,
          max_length=1024,
          temperature=0.7,
          do_sample=True
      )
    ###################################################
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    vanilla_responses.append(prediction)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
display(retrieved_node_scores)
display(test_queries)#[:5]

[[0.8347891343103313, 0.8329465845049433, 0.8307812303572081]]

['How do I make creme brulee?']

In order to answer the questions below, we'll need to examine the output from the RAG and vanilla systems

In [13]:
print("RAG system results with similarity scores ....")

for i, (texts, scores) in enumerate(zip(retrieved_node_texts, retrieved_node_scores)):
    print("---" * 100)
    print(f"----- Query {i} -----" )
    print(test_queries[i])
    for j, (text, score) in enumerate(zip(texts, scores)):
      print(f"----- Node {j} -----")
      print(text)
      print("score")
      print(score)

RAG system results with similarity scores ....
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----- Query 0 -----
How do I make creme brulee?
----- Node 0 -----
rachel khoo's creme brulee 

 none none, 300.0 milliliter cream, 200.0 milliliter milk, 1.0 vanilla pod, 6.0 egg yolks, 100.0 gram sugar, 30.0 gram white sugar, 30.0 gram brown sugar 

 in a saucepan, heat the cream and milk . cut the vanilla pod in half, and use a knife to remove seeds, and add to the milk and cream . bring to a boil, and remove from heat . separate the yolks from the whites, add sugar to the yolks, and gently add the hot cream, stirring gently . do not over mix to avoid creating too many bubbles . preheat the oven to 110 degrees celsius . divide the cream into

In [14]:
print("RAG and vanilla system responses to each query...")
for i, (rag_response, vanilla_response) in enumerate(zip(rag_responses, vanilla_responses)):
  print("---" * 100)
  print(f"----- Query {i} -----" )
  print(test_queries[i])
  print("----- RAG Response -----")
  print(rag_response)
  print("----- Vanilla Response -----")
  print(vanilla_response.replace(test_queries[i], "").strip())

RAG and vanilla system responses to each query...
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----- Query 0 -----
How do I make creme brulee?
----- RAG Response -----
1. Preheat your oven to 110°C (230°F). 2. In a saucepan, heat the cream and milk together. Cut the vanilla pod in half, remove the seeds, and add the chopped vanilla pod to the milk and cream. Bring the mixture to a boil, and remove it from heat. Separate the yolks from the whites, add sugar to the yolks, and gently stir the cream with the yolks and sugar. Do not overmix to avoid creating too many bubbles. Preheat the oven to 110°C (230°F). Divide the cream into six
----- Vanilla Response -----
What ingredients are needed, and how do you prepare the batter for a creamy,

### Questions - Answers

For questions 1 and 2, first qualitatively compare the output of the vanilla and RAG-based systems.


1. Do you observe differences between the quality of the RAG and vanilla responses? If yes, what are these?

A: Yes, RAG responses are more compact and to the point, whereas vanilla response is more detailed and contains more information than asked including Ingredients, Tips, Notes, Hashtags etc.


2.  Inspect the retrieved recipes and their scores. Do they make sense for the queries? Do the scores match your intuition about their relevance for the query?

A: As per the query, the retrieved recipes all have "creme brulee" in the title and describes the process of making "creme brulee". Their scores are 0.80 or higher - which makes sense intuitively.

3. Was there any benefit to using RAG for this use-case or could we have just used the vanilla system? Are there other use cases where we'd need to use RAG rather than relying on the base weights of an LLM?

A: In this case, if the ingredients are not a big necessity to be listed explicitly, then the RAG system avoided responding with excess information and avoided clutter. However, it also skipped over detailed instructions and only generated the summary version of the instructions, which may not be desired every time.


4.  What does the embedding model do? What is the measure used to score the relevance of retrieved documents?

A: The embedding model converts the query and to-be retrieved texts/recipes to be encoded into embedding space and find score based on how the query and to-be retrieved texts are semantically similar as per their embeddings.

Sources:

https://docs.llamaindex.ai/en/stable/api_reference/evaluation/semantic_similarity/

https://arxiv.org/pdf/2108.06130


